# **VizWiz Image Classification with InceptionV3**

### This notebook demonstrates how to perform image classification on the VizWiz dataset using a pre-trained InceptionV3 model from TensorFlow. It covers loading the pre-trained weights, saving and loading the model, and making predictions on new images.

---

This cell imports necessary libraries for building and training a TensorFlow model, including modules for file path manipulation, image handling, and numerical operations.

In [ ]:
# Imports

import tensorflow as tf
import pathlib
import os
import pathlib
import IPython
import json
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import time

# Note: Some imports might be redundant, consider cleaning them up if not used.

This cell downloads the pre-trained weights of the InceptionV3 model from the ImageNet dataset. InceptionV3 is a widely used convolutional neural network architecture for image classification.

In [ ]:
# Downloading the weights of the base model

# InceptionV3 model pre-trained on ImageNet
base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (299, 299, 3), # Expected input image shape
    include_top = True, # Include the fully-connected layer at the top
    weights = "imagenet", # Use weights pre-trained on ImageNet
    classes=1000, # Number of classes in ImageNet
    classifier_activation="softmax", # Activation function for the output layer
)

This cell saves the downloaded InceptionV3 base model to a specified file path in HDF5 format (`.h5`). This allows for later loading of the model without needing to re-download the weights.

In [ ]:
# Saving the downloaded base_model

# Define the path to save the model
saving_path = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/SavedBaseModel.h5')

# Save the model in HDF5 format
base_model.save(saving_path)

This cell loads the previously saved InceptionV3 model and sets its layers to be non-trainable. It also defines preprocessing and augmentation layers for image data, although the augmentation layers are currently commented out. The cell then defines the model's input and connects it to the base model, applying the InceptionV3 preprocessing. The commented-out code suggests an intention to potentially add more layers, compile the model, and use KerasTuner for hyperparameter tuning and training, but currently, it only defines the model structure and prints its summary.

In [ ]:
# Model Definition and Loading

# Define the path to the saved model
model_path = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/SavedBaseModel.h5')

# Loading base_model
base_model = tf.keras.models.load_model(model_path)

# Setting the base model as non-trainable to use it as a fixed feature extractor
base_model.trainable = False

# Define preprocessing layers
# Rescaling to (1, -1) range required for inceptionV3 model
rescaling = tf.keras.Sequential([
  tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
])

# Define augmentation layers (currently commented out)
# Applying augmentations to the images
augmentation = tf.keras.Sequential([
   tf.keras.layers.RandomFlip("horizontal"),
   tf.keras.layers.RandomRotation(0.1)
])

# def build_model(hp): # Commented out function for KerasTuner

# Define the model's input layer
inputs = tf.keras.Input(shape=(299, 299, 3))

# Apply preprocessing and pass through the base model
# x = augmentation(inputs) # Apply augmentation (commented out)
# x = rescaling(x) # Apply rescaling (commented out)
x = tf.keras.applications.inception_v3.preprocess_input(inputs) # Apply InceptionV3 specific preprocessing
x = base_model(x, training=False) # Pass through the base model in inference mode

# Add more layers (commented out)
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
# outputs = tf.keras.layers.Dense(1000, activation='softmax')(x) # "softmax" in the final layer for decision making

# Create the functional model
model = tf.keras.Model(inputs, x)

# learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log") # Commented out learning rate for KerasTuner

# Compile the model (commented out)
# model.compile(
#     optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005), # compiling with low learning rate
#     loss=tf.losses.SparseCategoricalCrossentropy(),
#     metrics=['accuracy'],
#     run_eagerly = True)

#  return model # Commented out return for KerasTuner

# KerasTuner setup (commented out)
# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="val_accuracy",
#     max_trials=3,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="/content/drive/MyDrive/Transfer",
#     project_name="Tuner",
# )

# tuner.search_space_summary() # Commented out KerasTuner summary

# tuner.search(dataset_path, epochs=5, validation_data = dataset_path_val) # Commented out KerasTuner search

# tuner.results_summary() # Commented out KerasTuner results

# Model fitting (commented out)
# model.fit( # fitting the whole model for non-trainable base
#     dataset_path,
#     epochs=1,
#     validation_data = dataset_path_val)
    # validation_data = dataset_path_val)
    #verbose = 1)

# Print the model summary
model.summary()

This cell defines paths to training, validation, and test data directories. It then compiles the loaded model with an Adam optimizer, sparse categorical crossentropy loss (assuming integer labels), and accuracy metric. The code then iterates through image files in the validation directory, loads each image, preprocesses it, makes a prediction using the model, and stores the predicted class index in a dictionary with the filename as the key. Finally, it prints the dictionary and saves it to a JSON file. A time delay is included every 600 predictions.

In [ ]:
# Predicting on Validation Data

# Define paths to data directories
train_data_path = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/trai/')
val_data_path = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/val/')
data_path_test = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/test/')

# Compile the loaded model for prediction
model.compile(                                                   # compiling the loaded model
    optimizer='adam', # Using Adam optimizer
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits= True), # Using SparseCategoricalCrossentropy loss
    metrics=['accuracy']) # Monitoring accuracy

# Initialize a dictionary to store predictions and a counter
dictionary = dict()
counter = 0

# Iterate through image files in the validation directory
for file in os.listdir(os.path.join(val_data_path)):
    image_path = os.path.join(val_data_path, file)

    # Load and preprocess the image
    image = tf.keras.utils.load_img(image_path,
                                    target_size=(299,299)) # Resize image to target size
    input_arr = tf.keras.utils.img_to_array(image) # Convert image to numpy array
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    # plt.imshow(image) # Display image (commented out)
    # plt.show() # Show plot (commented out)

    # Make a prediction
    predictions = model.predict(input_arr)
    pred = np.argmax(predictions, axis=1) # Get the index of the predicted class

    # Store the prediction in the dictionary
    dictionary[file] = int(pred[0])
    counter +=1

    # Introduce a time delay every 600 predictions
    if counter % 600 == 0:
        time.sleep(5) # Pause for 5 seconds

# Print the dictionary of predictions
print(dictionary)

# Writing to sample.json
# Open the JSON file in append mode and save the dictionary
with open("/home/samer/Desktop/Beedoo/VizWiz/sample_val.json", "a") as outfile:
    json_object = json.dump(dictionary, outfile)